In [ ]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType
import time
import json

spark = SparkSession.builder.appName("GetAllStockDataFromApiByDate").getOrCreate()

url = "https://api.polygon.io/v3/reference/tickers?market=stocks&active=true&order=asc&limit=1000&apiKey=VOYRq_vnd9soI0AhCO8FQpK4auZ6ppc3"

ticker_details_schema = StructType([
    StructField("active", StringType(), True),
    StructField("cik", StringType(), True),
    StructField("composite_figi", StringType(), True),
    StructField("currency_name", StringType(), True),
    StructField("last_updated_utc", StringType(), True),
    StructField("locale", StringType(), True),
    StructField("market", StringType(), True),
    StructField("name", StringType(), True),
    StructField("primary_exchange", StringType(), True),
    StructField("share_class_figi", StringType(), True),
    StructField("ticker", StringType(), True),
    StructField("type", StringType(), True)
])


response = requests.get(url)
ticker_details_df = spark.createDataFrame(response.json()["results"], schema = ticker_details_schema)

flag = 0

while(flag==0):
    time.sleep(30)
    if 'next_url' in response.json():
        next_url = response.json()['next_url'] + "&apiKey=VOYRq_vnd9soI0AhCO8FQpK4auZ6ppc3"
        response = requests.get(next_url)
        
        ticker_details_df = ticker_details_df.union(spark.createDataFrame(response.json()["results"], schema = ticker_details_schema))

    else:
        print("No data")
        flag = 1

ticker_details_df = ticker_details_df.coalesce(8)

ticker_details_df.write.format("parquet").\
    option("compression", "snappy").\
        mode("append").\
            save("gs://stocks-pipeline/raw-data/ticker_details")

spark.stop()
